# San Francisco Fire Calls
Questo notebook contiene una serie di esempi su come lavorare con DaFrame e SparkSQL per analizzare dati sul dataset del dipartimento dei vigili del fuoco di San Francisco.

Ispezionare la posizione del dataset (si trova in un bucket S3 pubblico)

In [0]:
%fs

ls /databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

path,name,size,modificationTime
dbfs:/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv,sf-fire-calls.csv,1137925359,1576280979000


Primissima ispezione dei dati, direttamente dalla loro posizione

In [0]:
%fs

head /databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

[Truncated to first 65536 bytes]
Call Number,Unit ID,Incident Number,CallType,Call Date,Watch Date,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,OrigPriority,Priority,Final Priority,ALS Unit,Call Type Group,NumAlarms,UnitType,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,"",1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,"",1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666
20110016,E03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,7,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E03,2.683333333333333
20110016,E38,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:17 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,1,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E38,2.1
20110016,E41,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,8,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E41,2.716666666666667
20110016,M03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:38 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,true,"",1,MEDIC,10,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-M03,2.7666666666666666
20110016,RS1,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:57 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,RESCUE SQUAD,9,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-RS1,3.2666666666666666
20110016,T03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,TRUCK,5,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T03,3.2333333333333334
20110016,T13,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:44 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,TRUCK,2,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T13,2.95
20110017,M13,2003236,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:27:14 AM,700 Block of FOLSOM ST,SF,94107,B03,01,2176,1,1,2,true,"",1,MEDIC,1,3,6,South of Market,"(37.7831757914875, -122.399603541822)",020110017-M13,6.933333333333334
20110019,M36,2003238,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:28:30 AM,9TH ST/LARKIN ST,SF,94103,B02,36,2337,1,1,2,true,"",1,MEDIC,1,2,6,South of Market,"(37.7774938436091, -122.41629170277)",020110019-M36,6.116666666666666
20110020,E38,2003239,Outside Fire,01/11/2002,01/10/2002,Other,01/11/2002 02:15:33 AM,3000 Block of WEBSTER ST,SF,94123,B04,16,3462,3,

Definisco la locazione del fine dentro una variabile

In [0]:
sf_fire_file = "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"

Voglio definire io lo schema e NON farglielo dedurre a Spark (facendo inferSchema=True). E' un file molto grande (4 milioni di righe) quindi la deduzione dello schema da parte di Spark sarebbe un processo molto dispendioso.

In [0]:
# importo i datatype

from pyspark.sql.types import *

In [0]:
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

In [0]:
# creo il DataFrame che legge il file, con lo schema che gli ho definito
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

Faccio il **cache** del DataFrame visto che performeremo diverse operazioni su di esso.

cache() è una **trasformazione** Apache Spark, memorizza nella cache il dataframe, il set di dati o RDD specificato nella memoria dei lavoratori del cluster. Proprio perche' e' una trasformazione, la sua memorizzazione avviene solo quando e' seguita da un'**azione**, ad esempio count(). Facendo il count() (che restituisce il conteggio di tutte le righe del DataFrame) vengono elaborate tutte le partizioni, quindi tutte le partizioni vengono memorizzate nella cache.

Quindi fare semplicemente df.cache() non memorizza l'operazione.

Doc: https://learn.microsoft.com/it-it/azure/databricks/kb/scala/best-practice-cache-count-take

In [0]:
fire_df.cache().count()

Out[6]: 4380660

### Prime esplorazioni

In [0]:
fire_df.count()

Out[7]: 4380660

In [0]:
fire_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [0]:
display(fire_df.limit(10))

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.3166666
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166667
20110016,E03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,ENGINE,7,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E03,2.6833334
20110016,E38,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:17 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,ENGINE,1,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E38,2.1
20110016,E41,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,ENGINE,8,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E41,2.7166667
20110016,M03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:38 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,true,null,1,MEDIC,10,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-M03,2.7666667
20110016,RS1,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:57 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,RESCUE SQUAD,9,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-RS1,3.2666667


Voglio fare fuori i record che corrispondono a "Medical Incident" come tipo di chiamata.

In [0]:
from pyspark.sql.functions import col

In [0]:
few_fire_df = (fire_df
              .select("IncidentNumber", "AvailableDtTm", "CallType")
              .where(col("CallType") != "Medical Incident"))

display(few_fire_df.limit(10))

IncidentNumber,AvailableDtTm,CallType
2003235,01/11/2002 01:47:00 AM,Structure Fire
2003235,01/11/2002 01:51:54 AM,Structure Fire
2003235,01/11/2002 01:47:00 AM,Structure Fire
2003235,01/11/2002 01:47:00 AM,Structure Fire
2003235,01/11/2002 01:51:17 AM,Structure Fire
2003235,01/11/2002 01:47:00 AM,Structure Fire
2003235,01/11/2002 01:46:38 AM,Structure Fire
2003235,01/11/2002 01:46:57 AM,Structure Fire
2003235,01/11/2002 01:47:00 AM,Structure Fire
2003235,01/11/2002 01:51:44 AM,Structure Fire


##### Q1) Quanti tipi di singole chiamate sono state fatte al dipartimento?
Per essere sicuro non conto i null.

In [0]:
display(fire_df.select("CallType").distinct().where(col("CallType").isNotNull()).count())

32

##### Q2) Quali sono i tipi di singole chiamate fatte al dipartimento?

In [0]:
display(fire_df.select("CallType").distinct().where(col("CallType").isNotNull()))

CallType
Elevator / Escalator Rescue
Marine Fire
Aircraft Emergency
Confined Space / Structure Collapse
Administrative
Alarms
Odor (Strange / Unknown)
Citizen Assist / Service Call
HazMat
Watercraft in Distress


##### Q3) Risposte dove c'e' stato un ritardo superiore a 5 minuti
1. rinominare il campo "Delay" con "ResponseDelayedinMins"
2. creare un nuovo DataFrame
3. Mostra solo il nuovo campo con ritardo superiore a 5 minuti

In [0]:
new_fire_df = (fire_df
               .withColumnRenamed("Delay", "ResponseDelayedinMins")
               .where(col("ResponseDelayedinMins") > 5)
              )

display(new_fire_df.select("ResponseDelayedinMins").limit(10))

ResponseDelayedinMins
5.233333
6.9333334
6.116667
7.85
77.333336
5.4166665
6.5
6.85
6.85
6.3333335


Faccio dell' ETL:

1. Trasforma le stringhe data in timestamp in modo tale da poterle utilizzare nel formato corretto
2. Restituisci la trasformazione in un nuovo DataFrame
3. Cacha il nuovo DataFrame

In [0]:
from pyspark.sql.functions import to_timestamp, to_date

In [0]:
fire_ts_df = (fire_df
              .withColumn("IncidentDate", to_date(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
              .withColumn("OnWatchDate",   to_date(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate")
              .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm"))


In [0]:
fire_ts_df.cache().count()

Out[17]: 4380660

Controllo le nuove colonne che abbiano il campo in modalita' timestamp

In [0]:
display(fire_ts_df.select("IncidentDate", "OnWatchDate", "AvailableDtTs").limit(5))

IncidentDate,OnWatchDate,AvailableDtTs
2002-01-11,2002-01-10,2002-01-11T01:58:43.000+0000
2002-01-11,2002-01-10,2002-01-11T02:10:17.000+0000
2002-01-11,2002-01-10,2002-01-11T01:47:00.000+0000
2002-01-11,2002-01-10,2002-01-11T01:51:54.000+0000
2002-01-11,2002-01-10,2002-01-11T01:47:00.000+0000


**Q4) Quali sono le chiamate piu' comuni?**

Ordinale in maniera decrescente

In [0]:
from pyspark.sql.functions import count

In [0]:
display((fire_ts_df
        .select("CallType")
        .where(col("CallType").isNotNull())
        .groupBy("CallType")
        .agg(count(col("CallType")).alias("NumCalls"))
        .orderBy("NumCalls", ascending=False)
        .limit(5))
       )

CallType,NumCalls
Medical Incident,2843475
Structure Fire,578998
Alarms,483518
Traffic Collision,175507
Citizen Assist / Service Call,65360


**Q4a) Quali sono i ZIP code piu' ricorrenti?**

Investighiamo su quali sono i zip code piu' ricorrenti e le loro relative chiamate.

1. Filter out by CallType
2. Group them by CallType and Zip code
3. Count them and display them in descending order

In [0]:
display((fire_ts_df
         .select("CallType", "ZipCode")
         .where(col("CallType").isNotNull())
         .groupBy("CallType", "Zipcode")
         .count()
         .orderBy("count", ascending=False)
         .limit(5))
       )

CallType,Zipcode,count
Medical Incident,94102,401457
Medical Incident,94103,370215
Medical Incident,94110,249279
Medical Incident,94109,238087
Medical Incident,94124,147564


**Q4b) Quali sono i quartieri di San Francisco che corrispondono a 94102 e 94103?**

Let's find out the neighborhoods associated with these two zip codes. In all likelihood, these are some of the contested 
neighborhood with high reported crimes.

In [0]:
display((fire_ts_df
        .select("Neighborhood", "Zipcode")
        .where((col("Zipcode") == 94102) | (col("Zipcode") == 94103))
        .distinct()
        .orderBy("Zipcode", "Neighborhood"))
    )

Neighborhood,Zipcode
Financial District/South Beach,94102
Hayes Valley,94102
Mission,94102
Nob Hill,94102
South of Market,94102
Tenderloin,94102
Western Addition,94102
Castro/Upper Market,94103
Financial District/South Beach,94103
Hayes Valley,94103


**Q5) Qante sono il numero di chiamate, il minimo, il massimo e la media di ritardi?**

Let's use the built-in Spark SQL functions to compute the sum, avg, min, and max of few columns:

* Number of Total Alarms
* What were the min and max the delay in response time before the Fire Dept arrived at the scene of the call

In [0]:
from pyspark.sql.functions import sum, avg, min, max

In [0]:
display((fire_ts_df.select(count("CallType").alias("NumAlarms")
                           ,avg("Delay").alias("AvgDelay")
                           ,min("Delay").alias("minDelay")
                           ,max("Delay").alias("MaxDelay")
                          )
        )
       )

NumAlarms,AvgDelay,minDelay,MaxDelay
4380660,3.902170335891614,0.016666668,1879.6167


**Q6a) Quanti anni distinti ci sono in questo file csv?**

We can use the `year()` SQL Spark function off the Timestamp column data type IncidentDate.

In [0]:
from pyspark.sql.functions import year

In [0]:
display((fire_ts_df
        .select(year("IncidentDate").alias("DistinctYears"))
        .distinct()
        .sort(col("DistinctYears").desc()))
        )
 

DistinctYears
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009


**Q6b) Quale settimana del 2018 ha avuto piu' chiamate?**

**Note**: Week 1 is the New Years' week and week 25 is the July 4 the week. Loads of fireworks, so it makes sense the higher number of calls.

In [0]:
from pyspark.sql.functions import weekofyear

In [0]:
display((fire_ts_df
        .where(year("IncidentDate") == 2018)
        .groupBy(weekofyear("IncidentDate").alias("WeekNumber"))
        .count()
        .sort(col("count").desc()))
       )

WeekNumber,count
1,6401
25,6163
13,6103
22,6060
44,6048
27,6042
16,6009
40,6000
43,5986
5,5946


**Q-7) Quali quartieri che hanno le risposte peggiori in termini di Delay?**

In [0]:
from pyspark.sql.functions import median

In [0]:
display((fire_ts_df
        .where(year("IncidentDate") == 2018)
        .groupBy("Neighborhood")
        .agg(median("Delay").alias("MedianDelay"), avg("Delay").alias("AvgDelay"))
        .orderBy(median("Delay"), ascending=False)
        .limit(5))
       )

Neighborhood,MedianDelay,AvgDelay
Treasure Island,3.9666666984558105,5.514422900772491
Presidio,3.933333396911621,4.898821295123589
Lincoln Park,3.591666579246521,5.50080411449859
None,3.5166666507720947,4.000947864982189
Twin Peaks,3.3333332538604736,4.2593417591104155


**Q-8a) Completo il flusso scrivendo il file in Parquet e salvare in una tabella**

In [0]:
%sql
create database if not exists FireDB

In [0]:
# remove the folder for the Parquet files
dbutils.fs.rm("/tmp/fireServiceParquet", True)
# writing the parquet file
fire_ts_df.write.format("parquet").mode("overwrite").save("/tmp/fireServiceParquet/")
# creating the table in FireDB database
fire_ts_df.write.format("parquet").mode("overwrite").saveAsTable("FireDB.FireServiceCalls")

In [0]:
%sql

-- cache della tabella
CACHE TABLE FireDB.FireServiceCalls

In [0]:
%sql
-- check della tabella creata
SELECT * FROM FireServiceCalls LIMIT 10

CallNumber,UnitID,IncidentNumber,CallType,CallFinalDisposition,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,ResponseDelayedinMins,IncidentDate,OnWatchDate,AvailableDtTS
111050354,E14,11034920,Medical Incident,Other,500 Block of 21ST AVE,SF,94121,B07,14,7171,3,3,3,true,null,1,ENGINE,1,7,1,Outer Richmond,"(37.7774255992901, -122.480311994328)",111050354-E14,4.7833333,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:27:08.000+0000
111050355,E03,11034921,Structure Fire,Other,HYDE ST/BUSH ST,SF,94109,B04,03,1561,3,3,3,true,null,1,ENGINE,1,4,3,Nob Hill,"(37.7891101748937, -122.417016879226)",111050355-E03,1.9166666,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:10:54.000+0000
111050355,T03,11034921,Structure Fire,Other,HYDE ST/BUSH ST,SF,94109,B04,03,1561,3,3,3,false,null,1,TRUCK,2,4,3,Nob Hill,"(37.7891101748937, -122.417016879226)",111050355-T03,2.4333334,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:10:54.000+0000
111050356,73,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,true,null,1,MEDIC,10,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-73,2.0666666,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:24:56.000+0000
111050356,B06,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,6,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-B06,2.6,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:22:46.000+0000
111050356,B10,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,4,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-B10,3.25,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:25:00.000+0000
111050356,D3,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,7,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-D3,3.5,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:23:01.000+0000
111050356,E29,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,true,null,1,ENGINE,8,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-E29,2.6,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:22:50.000+0000
111050356,E37,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,ENGINE,2,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-E37,2.6666667,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:25:10.000+0000
111050356,RS2,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,RESCUE SQUAD,5,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-RS2,3.05,2011-04-15T00:00:00.000+0000,2011-04-15T00:00:00.000+0000,2011-04-15T23:24:11.000+0000


**Q8c) Read data from the Parquet file?**

Note we don't have to specify the schema here since it's stored as part of the Parquet metadata

In [0]:
file_parquet_df = spark.read.format("parquet").load("/tmp/fireServiceParquet/")
display(file_parquet_df.limit(10))

CallNumber,UnitID,IncidentNumber,CallType,CallFinalDisposition,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay,IncidentDate,OnWatchDate,AvailableDtTS
111050354,E14,11034920,Medical Incident,Other,500 Block of 21ST AVE,SF,94121,B07,14,7171,3,3,3,true,null,1,ENGINE,1,7,1,Outer Richmond,"(37.7774255992901, -122.480311994328)",111050354-E14,4.7833333,2011-04-15,2011-04-15,2011-04-15T23:27:08.000+0000
111050355,E03,11034921,Structure Fire,Other,HYDE ST/BUSH ST,SF,94109,B04,03,1561,3,3,3,true,null,1,ENGINE,1,4,3,Nob Hill,"(37.7891101748937, -122.417016879226)",111050355-E03,1.9166666,2011-04-15,2011-04-15,2011-04-15T23:10:54.000+0000
111050355,T03,11034921,Structure Fire,Other,HYDE ST/BUSH ST,SF,94109,B04,03,1561,3,3,3,false,null,1,TRUCK,2,4,3,Nob Hill,"(37.7891101748937, -122.417016879226)",111050355-T03,2.4333334,2011-04-15,2011-04-15,2011-04-15T23:10:54.000+0000
111050356,73,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,true,null,1,MEDIC,10,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-73,2.0666666,2011-04-15,2011-04-15,2011-04-15T23:24:56.000+0000
111050356,B06,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,6,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-B06,2.6,2011-04-15,2011-04-15,2011-04-15T23:22:46.000+0000
111050356,B10,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,4,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-B10,3.25,2011-04-15,2011-04-15,2011-04-15T23:25:00.000+0000
111050356,D3,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,CHIEF,7,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-D3,3.5,2011-04-15,2011-04-15,2011-04-15T23:23:01.000+0000
111050356,E29,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,true,null,1,ENGINE,8,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-E29,2.6,2011-04-15,2011-04-15,2011-04-15T23:22:50.000+0000
111050356,E37,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,ENGINE,2,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-E37,2.6666667,2011-04-15,2011-04-15,2011-04-15T23:25:10.000+0000
111050356,RS2,11034922,Structure Fire,Other,1000 Block of POTRERO AVE,SF,94110,B10,07,2553,3,3,3,false,null,1,RESCUE SQUAD,5,10,10,Potrero Hill,"(37.7565080013216, -122.40654101432)",111050356-RS2,3.05,2011-04-15,2011-04-15,2011-04-15T23:24:11.000+0000
